In [192]:
import pandas as pd
import re
import string
import jieba
import collections

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [143]:
# all_question = pd.read_csv('./data/description.csv')
all_question = pd.read_excel('./data/description.xlsx')

In [144]:
all_question.head(20)

,Case Number,Description
0,1420656.0,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配..."
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）
2,1434184.0,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）
3,1436011.0,拿到的赫赛汀呈现块状是否正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用
5,1439604.0,赫赛汀最新说明书
6,1442983.0,赫赛汀哪里可以购买
7,1443005.0,有无过期赫赛汀输注到患者体内的疗效或安全性数据（患者怀疑医院将过期药品输注到体内，无产品批号信息）
8,1444557.0,赫赛汀注射前发现稀释液有结冰情况，能否使用，产品批号（SH0128，N3834/B3051）
9,1444573.0,赫赛汀用前发现稀释液有冰晶，能否使用


In [145]:
all_question.dropna(axis=0, inplace=True, subset=['Description'])

In [146]:
all_question.head(10)

,Case Number,Description
0,1420656.0,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配..."
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）
2,1434184.0,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）
3,1436011.0,拿到的赫赛汀呈现块状是否正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用
5,1439604.0,赫赛汀最新说明书
6,1442983.0,赫赛汀哪里可以购买
7,1443005.0,有无过期赫赛汀输注到患者体内的疗效或安全性数据（患者怀疑医院将过期药品输注到体内，无产品批号信息）
8,1444557.0,赫赛汀注射前发现稀释液有结冰情况，能否使用，产品批号（SH0128，N3834/B3051）
9,1444573.0,赫赛汀用前发现稀释液有冰晶，能否使用


In [147]:
all_question.isna().sum()

Case Number    0
Description    0
dtype: int64

In [148]:
all_question[pd.isnull(all_question).any(axis=1)]

,Case Number,Description


In [149]:
# 去除所有“[]”内的文本
def remove_special_string(text):
#     return text.replace("[Wechat]", "")
    return re.sub(r'\[[\s\S]*\]', '', text) 

In [150]:
all_question['Description'] = all_question.Description.apply(lambda x: remove_special_string(x))

In [151]:
# all_question.to_excel('./data/after_re_sub.xlsx')
all_question.head()

,Case Number,Description
0,1420656.0,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配..."
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）
2,1434184.0,请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）
3,1436011.0,拿到的赫赛汀呈现块状是否正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用


In [152]:
jieba.load_userdict('./jieba_dict.txt')

In [166]:
demo_wordcut = jieba.cut('赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。', cut_all=False)
print(' '.join(demo_wordcut).split())

['赫赛汀', '（', '批号', '1', '：', 'N3825B03B3049', ',', '批号', '2', '：', 'N3826B02B3050', '）', '配', '了', '之后', '结冰', '了', '，', '能否', '继续', '使用', '。']


In [156]:
len(all_question)

757

In [154]:
all_text_data = []
for idx,row in all_question.iterrows():
    all_text_data.append(row['Description'])

In [155]:
len(all_text_data)

757

In [185]:
chinese_punc = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."""
text = "赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。"
text.translate(str.maketrans('', '', chinese_punc))

'赫赛汀 批号1\xa0N3825B03B3049 批号2 N3826B02B3050 配了之后结冰了 能否继续使用'

In [186]:
chinese_punc = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."""
def jieba_cut(text):
    text = text.translate(str.maketrans('', '', chinese_punc))
    return ' '.join(jieba.cut(text, cut_all=False)).split()

In [187]:
# term-tfidf matrix
tfidf_vectorizer = TfidfVectorizer(input='content', tokenizer=jieba_cut, use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_text_data)

In [188]:
tfidf_matrix

<757x2119 sparse matrix of type '<class 'numpy.float64'>'
	with 10490 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_vectorizer.vocabulary_

In [198]:
kmeans_model = KMeans(n_clusters=30)
kmeans_model.fit(tfidf_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=30, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [199]:
clustering = collections.defaultdict(list)
for idx, label in enumerate(kmeans_model.labels_):
    clustering[label].append(idx)

In [203]:
for idx in clustering[0]:
    print(all_text_data[idx])

河北石家庄不到赫赛汀，怎么办
河北保定不到赫赛汀，怎么办
河北石家庄不到赫赛汀，怎么办
河北石家庄买不到赫赛汀，怎么办
陕西西安不到赫赛汀，怎么办
使用赫赛汀出现尿路感染怎么办？
赫赛汀河北石家庄什么时候进货
赫赛汀河北石家庄药店名称
咨询河北石家庄哪里可以购买赫赛汀
